# 003 List the names of all the actors who played in the movie Anand 1971

In [1]:
spark

org.apache.spark.sql.SparkSession@113f6c5d

In [2]:
import org.apache.spark.sql.functions._

In [3]:
import org.apache.spark.sql.types._

In [4]:
import org.apache.spark.sql.expressions.Window

In [5]:
val spark=SparkSession.builder().appName("movies app").master("local[*]").getOrCreate()

spark = org.apache.spark.sql.SparkSession@113f6c5d


org.apache.spark.sql.SparkSession@113f6c5d

In [6]:
import spark.implicits._

# UDFs

In [7]:
val getYear=udf((year:String)=>{
    val p="\\d{4}".r
    val yr=p.findFirstMatchIn(year).getOrElse("").toString
    if(yr!=""){
        yr.toInt
    }
    else{
        0
    }
})

getYear = UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))

# loading movies

In [8]:
val Movie=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Movie.csv")
.dropDuplicates()

Movie = [index: string, MID: string ... 4 more fields]


[index: string, MID: string ... 4 more fields]

# printing table structure to know what are the columns and it's data type

In [9]:
Movie.printSchema

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- num_votes: string (nullable = true)



# Displaying first two records to see sample data
# observation 1: year,rating,num_votes,index columns data types should be Integer
# observation 2: also we need do trim to remove any extra spaces

In [10]:
Movie.show(2)

+-----+---------+-----------------+----+------+---------+
|index|      MID|            title|year|rating|num_votes|
+-----+---------+-----------------+----+------+---------+
|   85|tt3224288|Beyond the Clouds|2017|   7.0|     1123|
|  913|tt2178508|   Son of Sardaar|2012|   4.0|     7895|
+-----+---------+-----------------+----+------+---------+
only showing top 2 rows



# displaying total records in the table

In [11]:
Movie.count

3475

# as per observation need to change the data type

In [12]:
val movies=Movie.withColumn("year",getYear(Movie("year")).cast(IntegerType))
.withColumn("title",trim(Movie("title")))
.withColumn("MID",trim(Movie("MID")))
.withColumn("rating",trim(Movie("rating")).cast(DoubleType))
.withColumn("num_votes",trim(Movie("num_votes")).cast(LongType))
.withColumn("index",trim(Movie("index")).cast(IntegerType))
.dropDuplicates("MID")

movies = [index: int, MID: string ... 4 more fields]


[index: int, MID: string ... 4 more fields]

# now print the schema to verify columns and it's data type

In [13]:
movies.printSchema

root
 |-- index: integer (nullable = true)
 |-- MID: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- rating: double (nullable = true)
 |-- num_votes: long (nullable = true)



# print the total number of records in the table after data cleaning

In [14]:
movies.count

3475

# show the first two records as a sample data

In [15]:
movies.show(2)

+-----+---------+--------------------+----+------+---------+
|index|      MID|               title|year|rating|num_votes|
+-----+---------+--------------------+----+------+---------+
| 3008|tt0036077|              Kismet|1943|   7.5|       71|
| 2969|tt0102701|Prahaar: The Fina...|1991|   7.9|     1696|
+-----+---------+--------------------+----+------+---------+
only showing top 2 rows



# Load the Person.csv file and it has header and store theses records in Person variable

In [16]:
val Person=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/Person.csv")

[index: string, PID: string ... 2 more fields]

Person = [index: string, PID: string ... 2 more fields]


# Print the schema of Person variable,first two records and it's count

In [17]:
Person.show(2)
Person.printSchema
val p1Count=Person.count

+-----+---------+---------------+------+
|index|      PID|           Name|Gender|
+-----+---------+---------------+------+
|    0|nm0000288| Christian Bale|  Male|
|    1|nm0000949| Cate Blanchett|Female|
+-----+---------+---------------+------+
only showing top 2 rows

root
 |-- index: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p1Count = 38285


38285

# Data Cleaning
# change the data types as per data and also remove extra spaces from data

In [18]:
val persons=Person.withColumn("PID",trim(Person("PID")))
.withColumn("Name",trim(Person("Name")))
.withColumn("Gender",trim(Person("Gender")))
.withColumn("index",trim(Person("index")).cast(IntegerType))
.dropDuplicates("PID")

persons = [index: int, PID: string ... 2 more fields]


[index: int, PID: string ... 2 more fields]

# show the sample data, schema and it's count of variable persons

In [19]:
persons.show(2)
persons.printSchema
val p2Count=persons.count

+-----+---------+-------------+------+
|index|      PID|         Name|Gender|
+-----+---------+-------------+------+
|15793|nm0022343|       Alpana|Female|
| 3121|nm0028416|Martin Andris|  Male|
+-----+---------+-------------+------+
only showing top 2 rows

root
 |-- index: integer (nullable = true)
 |-- PID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)



p2Count = 37566


37566

# print the difference between Person and persons records count

In [20]:
val diff=p1Count-p2Count
println(diff+" duplicates found")

719 duplicates found


diff = 719


719

# Loading and cleaning M_Cast table

In [21]:
val M_Cast=spark.read.option("header","true").csv("/home/use2cobadmin/practice_data/imdb/imdb_csv/M_Cast.csv")

M_Cast = [index: string, MID: string ... 2 more fields]


[index: string, MID: string ... 2 more fields]

# Sample data from M_Cast table

In [22]:
M_Cast.printSchema
M_Cast.show(2)
val m_c1Count=M_Cast.count

root
 |-- index: string (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: string (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|    0|tt2388771| nm0000288|  0|
|    1|tt2388771| nm0000949|  1|
+-----+---------+----------+---+
only showing top 2 rows



m_c1Count = 82837


82837

# Cleaning data m_cast

In [23]:
val m_cast=M_Cast.withColumn("ID",trim(col("ID")).cast(LongType))
.withColumn("index",trim(col("index")).cast(LongType))
.withColumn("MID",trim(col("MID")))
.withColumn("PID",trim(col("PID")))
.dropDuplicates("MID","PID")
//.dropDuplicates("MID","PID","ID")
.dropDuplicates("ID")
.dropDuplicates()

m_cast = [index: bigint, MID: string ... 2 more fields]


[index: bigint, MID: string ... 2 more fields]

# Sample data after data cleaning for m_cast

In [24]:
m_cast.printSchema
m_cast.show(2)
val m_c2Count=m_cast.count

root
 |-- index: long (nullable = true)
 |-- MID: string (nullable = true)
 |-- PID: string (nullable = true)
 |-- ID: long (nullable = true)

+-----+---------+----------+---+
|index|      MID|       PID| ID|
+-----+---------+----------+---+
|   26|tt2388771|nm10302077| 26|
|   29|tt2388771|nm10302080| 29|
+-----+---------+----------+---+
only showing top 2 rows



m_c2Count = 82837


82837

In [52]:
val anandMovieID=movies.filter($"title"==="Anand" && $"year"===1971).select("MID").first()(0)

anandMovieID = tt0066763


tt0066763

In [42]:
anandMovieID.show

+-----+---------+-----+----+------+---------+
|index|      MID|title|year|rating|num_votes|
+-----+---------+-----+----+------+---------+
|   66|tt0066763|Anand|1971|   8.8|    21616|
+-----+---------+-----+----+------+---------+



In [50]:
anandMovieID(0)

tt0066763

In [57]:
val anandMovieActors=persons.join(m_cast.filter($"MID"===anandMovieID),"PID").dropDuplicates().join(movies,"MID")
.select("MID","PID","year","Name","Gender")

anandMovieActors = [MID: string, PID: string ... 3 more fields]


[MID: string, PID: string ... 3 more fields]

In [66]:

anandMovieActors.orderBy("PID").show(10)


+---------+---------+----+----------------+------+
|      MID|      PID|year|            Name|Gender|
+---------+---------+----+----------------+------+
|tt0066763|nm0000821|1971|Amitabh Bachchan|  Male|
|tt0066763|nm0004435|1971|   Rajesh Khanna|  Male|
|tt0066763|nm0080231|1971|  Brahm Bhardwaj|  Male|
|tt0066763|nm0219946|1971|      Ramesh Deo|  Male|
|tt0066763|nm0219947|1971|       Seema Deo|Female|
|tt0066763|nm0222743|1971|      Dev Kishan|  Male|
|tt0066763|nm0451862|1971|     Durga Khote|Female|
|tt0066763|nm0474931|1971|   Lalita Kumari|Female|
|tt0066763|nm0667985|1971|    Lalita Pawar|Female|
|tt0066763|nm0695155|1971|    Atam Prakash|  Male|
+---------+---------+----+----------------+------+
only showing top 10 rows



In [67]:
anandMovieActors.count

17

# Validation 1

In [68]:

m_cast.filter($"MID"===anandMovieID).show


+-----+---------+---------+----+
|index|      MID|      PID|  ID|
+-----+---------+---------+----+
| 3662|tt0066763|nm0474931|3662|
| 3660|tt0066763|nm0222743|3660|
| 3655|tt0066763|nm0000821|3655|
| 3664|tt0066763|nm0080231|3664|
| 3665|tt0066763|nm1587173|3665|
| 3668|tt0066763|nm0802107|3668|
| 3658|tt0066763|nm0219947|3658|
| 3657|tt0066763|nm0219946|3657|
| 3669|tt0066763|nm0907888|3669|
| 3666|tt0066763|nm0667985|3666|
| 3659|tt0066763|nm0783996|3659|
| 3663|tt0066763|nm1783998|3663|
| 3654|tt0066763|nm0004435|3654|
| 3656|tt0066763|nm0764407|3656|
| 3670|tt0066763|nm1425570|3670|
| 3667|tt0066763|nm0451862|3667|
| 3661|tt0066763|nm0695155|3661|
+-----+---------+---------+----+



In [69]:
m_cast.filter($"MID"===anandMovieID).count

17